# Analisando as vendas da empresa
- Vamos utilizar a base disponível em:
    - https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce?select=olist_customers_dataset.csv
    - Essa é uma base de dados real, porém anonimizada
- Fizemos alguns tratamentos nos dados e disponibilizamos em 3 arquivos:
    - `base_vendas.xlsx`: todos os dados de venda por item da empresa, onde cada linha representa 1 item vendido
    - `base_pagamentos.xlsx`: base com as formas de pagamento usado naquela compra
    - `olist_order_reviews_dataset.csv`: base com o review dos pedidos, exatamente como baixada do Kaggle, onde cada linha representa a avaliação de uma pedido
- Uma base extra com o agrupamento das categorias também está disponível em `verifica_base.xlsx`

## Importando a base

In [ ]:
# Importando o pandas
import pandas as pd

In [ ]:
# Ignorando alguns erros que possam aparecer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importando a base de vendas do excel
vendas = pd.read_excel('base_vendas.xlsx')

In [ ]:
# Importando a base de grupos
grupos = pd.read_excel('verifica_base.xlsx')

In [ ]:
# Podemos unir essas duas bases utilizando o merge
vendas = pd.merge(
    vendas,
    grupos,
    how='left',
    left_on = 'product_category_name', right_on = 'categoria'
)

In [ ]:
# E também apagar as colunas que não estamos usando
vendas = vendas.drop(['product_category_name','categoria'],axis=1)

In [ ]:
# Verificando as informações da tabela
vendas.info()

### Tratando valores vazios e outliers

In [ ]:
vendas.isnull().sum()

**Retirando valores nulos das vendas**

In [ ]:
# Retirando os valores onde o preço é vazio
vendas = vendas[vendas.price.notnull()]

**Desconsiderando as vendas canceladas**

In [ ]:
# Retirando as vendas que foram canceladas
vendas = vendas[vendas.order_status != 'canceled']

**Retirando os outliers das datas**

In [ ]:
# Retirando a black friday
import datetime as dt
vendas = vendas[vendas.order_purchase_timestamp != dt.datetime(2017,11,24)]

In [ ]:
# Retirando o dia após a black friday
vendas = vendas[vendas.order_purchase_timestamp != dt.datetime(2017,11,25)]

**Atualizando categorias vazias como outros**

In [ ]:
# Substituindo valores vazios no grupo por nulo
vendas.loc[vendas.grupo.isnull(),'grupo'] = 'outros'

**Verificando valores vazios**

In [ ]:
# Contando quantos valores vazios existem em cada coluna
vendas.isnull().sum()

In [ ]:
# Entendendo as 15 colunas com order_approved_at vazias
vendas[vendas.order_approved_at.isnull()]

In [ ]:
# Apagando essas colunas
vendas = vendas.drop('order_approved_at',axis=1)

**Verificando os outliers**

In [ ]:
# Criando um boxplot
vendas.groupby('order_purchase_timestamp')['price'].sum().plot.box();

**Podemos exportar essa base já tratada para o excel**
- https://pandas.pydata.org/docs/getting_started/

In [ ]:
# Exportando para o Excel

# Qual pergunta eu quero responder?

### <font color='blue'>Qual vai ser a venda valor nos próximos dias?</font>

In [ ]:
vendas.head(3)

In [ ]:
# Visualizando a venda pelo tempo
vendas.groupby('order_purchase_timestamp')['price'].sum().plot();

In [ ]:
# Importando o matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Podemos criar uma nova variável para a venda por dia
venda_vlr = vendas.groupby('order_purchase_timestamp')['price'].sum()

In [ ]:
# E então traçar o gráfico dessa variável
fig, ax = plt.subplots(figsize=(12,6))

x = venda_vlr.index
y = venda_vlr.values

ax.plot(x, y, linewidth=2.0)

plt.show()

In [ ]:
venda_vlr.index[-1]

<a id='grafico_inicial'></a>
**Podemos traçar um gráfico para verificar como ficou a nossa estimativa**

In [ ]:
# Eu posso traçar uma reta qualquer para estimar a venda

fig, ax = plt.subplots(figsize=(12,6))

x = venda_vlr.index
y = venda_vlr.values

ax.plot(x, y, linewidth=2.0)

x_reta = [venda_vlr.index[0],venda_vlr.index[-1]]
y_reta = [5000,35000]

ax.plot(x_reta, y_reta,'--r')

plt.show()

In [ ]:
# Podemos guardar essa previsão em uma variável no nosso DataFrame

## Qual é a melhor forma de traçar essa reta?
- Utilizando a regressão linear do Scikit-Learn
    - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
# Ajustando a venda_vlr como um DataFrame
venda_vlr = venda_vlr.reset_index()
venda_vlr

In [ ]:
# Criando uma regressao

In [ ]:
# Antes de usar o Scikit-Learn, vamos tratar a data para conseguir colocar no modelo
# Importando o datetime
import datetime as dt

In [ ]:
venda_vlr.tail(3)

In [ ]:
# Fazendo a diferença entre o dia atual e o menor dia
venda_vlr['ajuste_data'] = (venda_vlr.order_purchase_timestamp - venda_vlr.order_purchase_timestamp.min()).dt.days

In [ ]:
# Importando a regressão linear do Scikit-Learn
from sklearn.linear_model import LinearRegression

In [ ]:
# Separando em X e y nossos dados
X = venda_vlr.ajuste_data.values.reshape(-1,1)
y = venda_vlr.price

In [ ]:
# Colocando nossos dados no modelo
reg = LinearRegression().fit(X, y)

In [ ]:
# Verificando o score
reg.score(X, y)

In [ ]:
# o coeficiente angular dessa reta
reg.coef_

In [ ]:
# e o coeficiente linear
reg.intercept_

In [ ]:
venda_vlr.head(3)

In [ ]:
# Podemos adicionar essa reta no gráfico que criamos
fig, ax = plt.subplots(figsize=(12,6))

x = venda_vlr.ajuste_data
y = venda_vlr.price

ax.plot(x, y, linewidth=2.0)

x_reta = venda_vlr.ajuste_data
y_reta = x_reta*reg.coef_[0] + reg.intercept_

ax.plot(x_reta, y_reta,'--r')

plt.show()

<a id="vendas_grupo"></a>
### <font color='blue'>Na verdade, eu quero saber qual a venda de determinados grupo</font>

In [ ]:
# Visualizando novamente a base de vendas
vendas.head(3)

In [ ]:
# Agrupando a venda por grupo
venda_grupos = vendas.groupby(['order_purchase_timestamp','grupo'])['price'].sum().reset_index()

In [ ]:
# Visualizando o plot de alguns grupos
venda_grupos.loc[venda_grupos.grupo == 'carro','price'].plot();

In [ ]:
# Agrupando a venda por data e grupo
venda_grupos = vendas.groupby(['order_purchase_timestamp','grupo'])['price'].sum().reset_index()

In [ ]:
# Verificando a data máxima
venda_grupos.order_purchase_timestamp.max()

In [ ]:
# E o shape desse DataFrame
venda_grupos.shape

In [ ]:
# Podemos então separar os nossos dados em treino e teste
treino = venda_grupos[venda_grupos.order_purchase_timestamp <= dt.datetime(2018,3,1)]
teste = venda_grupos[venda_grupos.order_purchase_timestamp > dt.datetime(2018,3,1)]

# Verificando o tamanho da base de treino
print(treino.shape)

# Verificando o tamanho da base de teste
print(teste.shape)

# Verificando a proporção de teste da nossa base
print(teste.shape[0]/venda_grupos.shape[0])

In [ ]:
# Podemos então visualizar essa base de treino e de teste
fig, ax = plt.subplots(figsize=(12,6))

filtro = 'beleza'

base_treino = treino[treino.grupo == filtro]
base_teste = teste[teste.grupo == filtro]

ax.plot(base_treino.order_purchase_timestamp, base_treino.price)
ax.plot(base_teste.order_purchase_timestamp, base_teste.price,'--r')

plt.show()

## Criando um modelo para estimar a venda desses pontos

In [ ]:
# Visualizando a base de treino
treino.head(3)

In [ ]:
# Separando X e y
X_treino = treino[['order_purchase_timestamp','grupo']]
y_treino = treino.price

In [ ]:
# Fazendo a regressão para esses dados
reg = LinearRegression().fit(X_treino, y_treino)

**O primeiro erro indica que não podemos passar um `Timestamp` para o modelo** <br>
- Vamos então fazer o tratamento de datas que já mostramos anteriormente

In [ ]:
# Utilizando o mesmo tratamento que fizemos na data
venda_grupos['ajuste_data'] = (venda_grupos.order_purchase_timestamp - venda_grupos.order_purchase_timestamp.min()).dt.days

treino = venda_grupos[venda_grupos.order_purchase_timestamp <= dt.datetime(2018,3,1)]
teste = venda_grupos[venda_grupos.order_purchase_timestamp > dt.datetime(2018,3,1)]

In [ ]:
# Novamente separando X e y
X_treino = treino[['ajuste_data','grupo']]
y_treino = treino.price

In [ ]:
# Novamente fazendo a regressão
reg = LinearRegression().fit(X_treino, y_treino)

**Já o segundo erro indica que também não podemos ter `valores em texto`** <br>
`ValueError: could not convert string to float`

- Para conseguir resolver, podemos usar o `.get_dummies()` do próprio pandas
    - https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

In [ ]:
# Fazendo o get_dummies para a base de treino
treino = pd.concat([treino,pd.get_dummies(treino.grupo)],axis=1)

In [ ]:
# Separando X e y
X_treino = treino.drop(['order_purchase_timestamp','grupo','price'],axis=1)
y_treino = treino.price

In [ ]:
# E fazendo a regressão
reg = LinearRegression().fit(X_treino, y_treino)

In [ ]:
# Verificando o score
reg.score(X_treino, y_treino)

In [ ]:
# o coeficiente angular
reg.coef_

In [ ]:
# e o coeficiente linear
reg.intercept_

In [ ]:
# Salvando na base de treino a previsao que acabamos de fazer
treino['previsao'] = reg.predict(X_treino)

In [ ]:
# Podemos então visualizar essa base de treino e de teste
fig, ax = plt.subplots(figsize=(12,6))

filtro = 'beleza'

base_treino = treino[treino.grupo == filtro]
base_teste = teste[teste.grupo == filtro]

ax.plot(base_treino.order_purchase_timestamp, base_treino.price)
ax.plot(base_teste.order_purchase_timestamp, base_teste.price)
ax.plot(base_treino.order_purchase_timestamp,base_treino.previsao,'--r')

plt.show()

### Será que meu modelo aprendeu ou apenas decorou?

In [ ]:
# Verificando a base de teste
teste.head(3)

In [ ]:
# E a transformação do grupo
teste = pd.concat([teste,pd.get_dummies(teste.grupo)],axis=1)

In [ ]:
# Separando X e y
X_teste = teste.drop(['order_purchase_timestamp','grupo','price'],axis=1)
y_teste = teste.price

In [ ]:
# E fazendo a previsão para a base de teste
teste['previsao'] = reg.predict(X_teste)

In [ ]:
# Podemos visualizar de forma gráfica essa previsão
fig, ax = plt.subplots(figsize=(12,6))

filtro = 'beleza'

base_treino = treino[treino.grupo == filtro]
base_teste = teste[teste.grupo == filtro]

ax.plot(base_treino.order_purchase_timestamp, base_treino.price)
ax.plot(base_teste.order_purchase_timestamp, base_teste.price)
ax.plot(base_treino.order_purchase_timestamp,base_treino.previsao,'--r')
ax.plot(base_teste.order_purchase_timestamp,base_teste.previsao,'--r')

plt.show()

### Podemos usar qualquer um dos nossos dados, desde que tratados, para colocar no modelo

In [ ]:
# Verificando novamente a base de venda

**No [primeiro exemplo](#vendas_grupo), agrupamos pela data completa da compra**

In [ ]:
# Agora vamos utilizar as colunas mes_compra e ano_compra

In [ ]:
# Separando em treino e teste

In [ ]:
# Agora vamos criar novamente o nosso modelo

# Separando X e y para treino

# Fazendo o fit do modeo               

# Fazendo a previsão para a base de treino

# Fazendo a previsão para a base de teste

In [ ]:
# Visualizando de forma gráfica

**Podemos usar esses valores previstos por categoria para fazer a previsão diária** <br>
**Será que isso melhorara nosso resultado?** <br>
**Vamos comparar ao [gráfico que vimos anteriormente](#grafico_inicial)**

In [ ]:
# Podemos agrupar esses valores por dia

In [ ]:
# E também visualizar graficamente

In [ ]:
# Adicionando a reta que fizemos anteriormente

## Testando a Árvore de Regressão
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

In [ ]:
# Verificando a base de treino

In [ ]:
# Separando X e y de treino

In [ ]:
# Importando o DecisionTreeRegressor

In [ ]:
# Criando o nosso regressor

In [ ]:
# Fazendo o fit do modelo

In [ ]:
# Verificando o score

In [ ]:
# Fazendo a previsão pro treino e teste

In [ ]:
# Adicionando esses dados no gráfico anterior

**O próprio Scikit-Learn te ajuda a analisar os erros**
- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html

In [ ]:
# Importando o erro médio absoluto

In [ ]:
# Analisando o erro na regressao

In [ ]:
# Analisando o erro na regressao

## Agora testando o Support Vector Regression
- https://scikit-learn.org/stable/modules/svm.html#regression <br><br><br><br>

...na próxima aula!